# Проект по SQL. Анализ книжного рынка.

В наши задачи входило провести анализ книжного рынка и найти ответы на несколько краеугольных вопросов, которые определят существование не только книжного рынка как такового, но и, без преувеличения, и всей письменной речи на тысячи лет вперед и на не меньшее число лет назад. 

Нам было предоставлено 5 таблиц внутри одной схемы: таблицы с перечнями книг, авторов, рейтингов, отзывов и издательств  и ER-диаграмма.

Ход исследования:

Настройка подключения к БД

Вывод таблиц

Создание запросов к таблицам

Выводы


## Настройка подключения к БД

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [3]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Вывод таблиц

Подготовим функцию:

In [4]:
def execute_sql_query(query):
    with engine.connect() as con:
        result = pd.read_sql(query, con)
    return result

# Пример использования функции
#query = '''SELECT * FROM books LIMIT 5'''
#result_df = execute_sql_query(query)

In [5]:
# Список таблиц, для которых нужно выполнить запрос
table_names = ['books', 'authors', 'publishers', 'ratings', 'reviews']

# Шаблон запроса 
query_template = '''SELECT * FROM {}'''

# Создаем переменные с именами таблиц и сохраняем результаты запросов в них
for table_name in table_names:
    query = query_template.format(table_name)
    globals()[table_name] = execute_sql_query(query)

# Выводим первые 5 строк каждой таблицы на экран
for table_name in table_names:
    print(f"Таблица: {table_name}")
    display(globals()[table_name].head())
    print("-" * 40)


Таблица: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


----------------------------------------
Таблица: authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


----------------------------------------
Таблица: publishers


...

----------------------------------------
Таблица: ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


----------------------------------------
Таблица: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


----------------------------------------


## Создание запросов к таблицам

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [6]:
query = '''

Select count(*)
From books
WHERE CAST(DATE_TRUNC('day', publication_date) AS date) > '2000-01-01';

'''
result1 = execute_sql_query(query)

In [7]:
result1

,count
0,819


После 1 января 2000 года вышло 819 книг

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [8]:
query = '''

SELECT
    b.book_id,
    b.title,
    (
        SELECT COUNT(*)
        FROM reviews AS r
        WHERE r.book_id = b.book_id
    ) AS review_count,
    (
        SELECT AVG(rt.rating)
        FROM ratings as rt
        WHERE rt.book_id = b.book_id
    ) AS average_rating
FROM
    books AS b


'''
result21 = execute_sql_query(query)

In [9]:
result21.head()

,book_id,title,review_count,average_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000


Для каждой книги были посчитаны число обзоров и средний рейтинг

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [10]:
query = '''

SELECT
    p.publisher_id,
    p.publisher,
    COUNT(*) AS book_count
FROM
    books AS b
INNER JOIN
    publishers AS p
ON
    b.publisher_id = p.publisher_id
WHERE
    b.num_pages > 50
GROUP BY
    p.publisher_id, p.publisher
ORDER BY
    book_count DESC
LIMIT 1;




'''
result3 = execute_sql_query(query)

In [11]:
result3

,publisher_id,publisher,book_count
0,212,Penguin Books,42


Было определено издательство, издавшее наибольшее число книг толще 50 страниц, это Penguin Books	

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [14]:
query = '''

SELECT authors.author_id, authors.author, AVG(avg_rating) AS avg_author_rating
FROM authors
JOIN books ON authors.author_id = books.author_id
JOIN (
    SELECT book_id, AVG(rating) AS avg_rating
    FROM ratings
    WHERE book_id IN (
        SELECT book_id
        FROM ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) >= 50
    )
    GROUP BY book_id
) AS book_avg_ratings ON books.book_id = book_avg_ratings.book_id
GROUP BY authors.author_id
ORDER BY avg_author_rating DESC
LIMIT 1;





'''
result43 = execute_sql_query(query)

In [15]:
result43

,author_id,author,avg_author_rating
0,236,J.K. Rowling/Mary GrandPré,4.283844


По выборке наиболее оцениваемых книг (с 50 и более рейтингами) был отобран автор книг с самым высоким средним рейтингом, это J.K. Rowling в сотрудничестве с иллюстратором Mary GrandPré со средним рейтингом книг 4.28

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [17]:
query = '''


WITH UsersRatings AS (
    SELECT
        r.username,
        COUNT(r.rating_id) AS rating_count
    FROM
        ratings AS r
    GROUP BY
        r.username
    HAVING
        COUNT(r.rating_id) > 48
)
SELECT
    AVG(COALESCE(rc.review_count, 0)) AS average_review_count
FROM
    UsersRatings AS ur
LEFT JOIN (
    SELECT
        username,
        COUNT(review_id) AS review_count
    FROM
        reviews
    GROUP BY
        username
) AS rc
ON
    ur.username = rc.username;



'''
result5 = execute_sql_query(query)

In [18]:
result5

,average_review_count
0,24.0


Было определено среднее число обзоров от пользователей, поставивших не менее 48 оценок, это 24 обзора

## Выводы


Перед нами была поставлена задача провести анализ книжного рынка и найти ответы на несколько краеугольных вопросов, которые определят существование не только книжного рынка как такового, но и, без преувеличения, и всей письменной речи на тысячи лет вперед и на не меньшее число лет назад.

Нам было предоставлено 5 таблиц внутри одной схемы: таблицы с перечнями книг, авторов, рейтингов, отзывов и издательств и ER-диаграмма.

Таблицы были выведены и визуально проинспектированы.

В результате сформулированных запросов нами была получена следующая бесценная информация:

1. После 1 января 2000 года вышло 819 книг

2. Издательством, издавшим наибольшее число книг толще 50 страниц, является Penguin Books,	42 изданные книги, не считая брошюр!

3. По выборке наиболее оцениваемых книг (с 50 и более рейтингами) был отобран автор книг с самым высоким средним рейтингом, это J.K. Rowling в сотрудничестве с иллюстратором Mary GrandPré со средним рейтингом книг 4.28

4. Было определено среднее число обзоров от пользователей, поставивших не менее 48 оценок, это 24 обзора








